In [1]:
import telegram.ext
from telegram.ext import Updater, CommandHandler
from telegram.ext import MessageHandler, Filters
import pandas as pd
import json
import numpy as np
from binance.client import Client
import os
import matplotlib.pyplot as plt
from jupyterthemes import jtplot
jtplot.style()

# binance setup
binance_api_key = os.environ.get("BINANCE_KEY")
binance_api_secret = os.environ.get("BINANCE_SECRET")
client = Client(binance_api_key, binance_api_secret)

# telegram setup
telegram_token = os.environ.get("TELEGRAM_TOKEN")
updater = Updater(telegram_token, use_context=True)
bot = updater.bot
job = updater.job_queue

ma_1_interval = 7
ma_2_interval = 14

def algo_bot (update):
    prices = []
    for kline in client.get_historical_klines_generator("BTCUSDT", Client.KLINE_INTERVAL_30MINUTE, "10 hour ago UTC"):
        float_result = float(kline[4])
        prices.append(float_result)
    
    ma_1 = sum(prices[-ma_1_interval:])/ma_1_interval
    print(ma_1)
    
    ma_2 = sum(prices[-ma_2_interval:])/ma_2_interval
    print(ma_2)
    
    state = ma_1 > ma_2
    state_temp = False
    
    if state != state_temp:
        state_temp = state
        if state:
            updater.bot.send_message(chat_id="@bebelere_balon", text= "Buy!")
        else:
            updater.bot.send_message(chat_id="@bebelere_balon", text= "Sell! \n Moving Average {0} Days: {1} \n Moving Average {2} Days: {3}"
                                     .format(ma_1_interval,ma_1,ma_2_interval,ma_2))
    else:
        updater.bot.send_message(chat_id="@bebelere_balon", text="No change.")

updater.job_queue.run_repeating(algo_bot, 60, first=0)
updater.start_polling()
updater.idle()



In [ ]:
prices = []
for kline in client.get_historical_klines_generator("BTCUSDT", Client.KLINE_INTERVAL_30MINUTE, "10 day ago UTC"):
    float_result = float(kline[4])
    prices.append(float_result)

ma_50_list = []
ma_50_result = []
ma_200_list = []
ma_200_result = []

graph_interval = len(prices)-1
print(graph_interval)

interval_1 = 7
interval_2 = 14
myrange_50 = graph_interval-interval_1 #430

invest_state = "idle"
state_temp = ""
state_new = ""

for i in range(graph_interval, graph_interval - interval_1,-1):
    print(i)
    ma_50_list.append(prices[i])

print(ma_50_list)
current_MA_small = sum(ma_50_list)/interval_1
print("MA_small: ", current_MA_small)

for i in range(graph_interval, graph_interval - interval_2, -1):
    ma_200_list.append(prices[i])
    
current_MA_large = sum(ma_200_list)/interval_2
print("MA_large: ", current_MA_large)

if current_MA_large > current_MA_small:
    invest_state = "short"
elif current_MA_large < current_MA_small:
    invest_state = "long"
else:
    invest_state = "idle"

#if invest_state == "short":
#    if shift_state:
#        shift_state = False
#        print("Sell")
#elif invest_state == "long":
#    if shift_state:
#        print("Buy")
#else:
#    print("State is idle")

for x in range(interval_1):
    ma_50_result.append(0)

for i in range(graph_interval-interval_1):
    for j in range(i, i + interval_1):
        ma_50_list.append(prices[j])
    avg = (sum(ma_50_list))/interval_1
    ma_50_result.append(avg)
    ma_50_list = []
    
for x in range(interval_2):
    ma_200_result.append(0)
    
for i in range(graph_interval-interval_2):
    for j in range(i, i + interval_2):
        ma_200_list.append(prices[j])
    avg = (sum(ma_200_list))/interval_2
    ma_200_result.append(avg)
    ma_200_list = []


plt.plot(prices, label="Price")
plt.plot(ma_50_result, label="MA50")
plt.plot(ma_200_result, label="MA200")

#plt.xlim(interval_2,graph_interval)
#plt.ylim(8500,9500)

plt.axis([interval_2,graph_interval,8000,9500])

In [ ]:
#backtest
state = bool
state_temp = False

for x in range(interval_2+1, graph_interval):
    state = ma_50_result[x] > ma_200_result[x] # TRUE = LONG, FALSE = SHORT
    if state != state_temp:
        state_temp = state
        if state:
            print("Buy")
        else:
            print("Sell")
    else:
        print("No change in state")
